pro repo brauche ich ein .json mit folgenden key-value-pairs  
- repo_name
- repo_owner
- readme
- requirements
- license
- source_code (.py & .ipynb)
- source_code_comments (.py & .ipynb)

alle .py & .ipynb files sollen zu einem string zusammengefasst werden  
ich will keinen coden von zusätzlichen packages verwenden! das wird sonst riesig  

ich gehe durch alle files vom repo und concatenate die entsprechend files in einem string  
danach cleaning




In [44]:
import pandas as pd
import json
import numpy as np
import os
from functools import reduce
import re

In [45]:
def read_md(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
                md_content = file.read()
    return md_content

In [46]:
repo_path = '../data/repo_unzip/abi-screenshot-to-code-ff18ae0'
input_data_path = '../data/input_data'

repo_parts = repo_path.split('/')[-1].split('-')

repo_name = '-'.join(reduce(lambda x, y: x + '-' + y, repo_parts[:-1]).split('-')[1:])
repo_owner = repo_path.split('/')[3].split('-')[0]

In [47]:
print(repo_name)
print(repo_owner)

screenshot-to-code
abi


In [48]:
tmp_json = {
    'repo_owner': repo_owner,
    'repo_name': repo_name,
    'requirements': None,
    'readme': None,
    'license': None,
    'source_code_comments': None,
    'source_code': None,
    'source_code_cleaned_comments': None,
    'source_code_cleaned': None
}

In [49]:
check_readme = False
check_license = False
check_requirements = False
list_source_code = []

In [50]:
# for root, dirs, files in os.walk(repo_path):
#     # Process the files first
#     print("Processing files at the top level:")
#     for file in files:
#         file_path = os.path.join(root, file)
#         print(f"File: {file_path}")

#     # Then process the directories
#     print("\nProcessing directories at the top level:")
#     for directory in dirs:
#         dir_path = os.path.join(root, directory)
#         print(f"Directory: {dir_path}")
    

In [51]:

exclude_dirs = ('dist', 'env', 'venv', 'vendor', 'third_party', 'tmp', 'temp', 'logs', 'assets', 'static', 'build', 'tests', 'docs', 'documentation') 

for root, dirs, files in os.walk(repo_path): # root: current directory path | dirs: list of subdirectories in the current path | files: list of files in the current directory
    # filters directories which are listed in exclude_dirs from dirs
    dirs[:] = [d for d in dirs if d not in exclude_dirs and not d.startswith('.')] # dirs[:]: update the list (list from os.walk()) dirs instead of creating a new one (generated with Microsoft Copilot)
    # print(f'root: {root}')
    # print(f'dirs: {dirs}')
    # print(f'files for root {root}: {files}')

    for file in files: # iterate through all files in files list
        try:
            file_path = os.path.join(root, file)
            if file.lower() in ('readme.md', 'readme') and check_readme is False:
                tmp_json['readme'] = read_md(file_path)
                check_readme = True
            elif file.lower() in ('license.md', 'license') and check_license is False:
                tmp_json['license'] = read_md(file_path)
                check_license = True
            elif file == 'requirements.txt' and check_requirements is False:
                with open(file_path) as data:
                    requirements_content = data.readlines()
                    tmp_json['requirements'] = ''.join(line.strip() for line in requirements_content)
                    check_requirements = True
            elif file.endswith(('.py', '.ipynb')):
                with open(file_path, 'r', errors='ignore') as f:
                    # list_source_code.append((file_path, f.read())) # to check if all files are in list_source_code add file_path
                    list_source_code.append(f.read())
                # print(f'from {file_path} the file: {file} was added to list_source_code')
        except Exception as e:
            print(f'Error during processing file {file_path}: {e}')

print(f'last file was {file_path}')

source_code_comments = ''.join(line.strip() for line in list_source_code)
tmp_json['source_code_comments'] = source_code_comments

last file was ../data/repo_unzip/abi-screenshot-to-code-ff18ae0/frontend/src/hooks/useThrottle.ts


In [52]:
len(list_source_code)

40

In [53]:
tmp_json['readme']

'# screenshot-to-code\n\n**NEW! 🚀 Interested in building full-stack apps with backends and databases using AI?** [Beta test our new tool for FREE.](https://fullyprivatechat.com/)\n\nA simple tool to convert screenshots, mockups and Figma designs into clean, functional code using AI. Now supporting Claude Sonnet 3.7!\n\nhttps://github.com/abi/screenshot-to-code/assets/23818/6cebadae-2fe3-4986-ac6a-8fb9db030045\n\nSupported stacks:\n\n- HTML + Tailwind\n- HTML + CSS\n- React + Tailwind\n- Vue + Tailwind\n- Bootstrap\n- Ionic + Tailwind\n- SVG\n\nSupported AI models:\n\n- Claude Sonnet 3.7 - Best model!\n- GPT-4o - also recommended!\n- DALL-E 3 or Flux Schnell (using Replicate) for image generation\n\nSee the [Examples](#-examples) section below for more demos.\n\nWe also just added experimental support for taking a video/screen recording of a website in action and turning that into a functional prototype.\n\n![google in app quick 3](https://github.com/abi/screenshot-to-code/assets/23818/

In [54]:
# print(len(tmp_json['source_code']))
# print(len(tmp_json['source_code_comments']))

In [55]:
def remove_comments(text):
    single_line_comments_pattern = r'#.*' # (generated with Microsoft Copilot)
    mulit_line_comments_pattern = r"\"\"\".*?\"\"\"|'''.*?'''" # (generated with Microsoft Copilot)
    cleaned_str = re.sub(single_line_comments_pattern, '', text)
    cleaned_str = re.sub(mulit_line_comments_pattern, '', cleaned_str, flags=re.DOTALL) # (generated with Microsoft Copilot)

    return cleaned_str


def remove_space_newline(text):
    cleaned_str = text.replace(' ', '').replace('\n', '')

    return cleaned_str


def remove_urls(text):
  
    url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+' # (source: https://github.com/souradipp76/ReadMeReady --> scripts/data.ipynb)
    cleaned_str = re.sub(url_pattern, '', text)
  
    return cleaned_str


def remove_html(text):
    html_pattern = r'<.*?>' # (source: https://github.com/souradipp76/ReadMeReady --> scripts/data.ipynb)
    cleaned_str = re.sub(html_pattern, '', text)
  
    return cleaned_str


def remove_emoji(text): # (source: https://github.com/souradipp76/ReadMeReady --> scripts/data.ipynb)
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols 
                           u"\U0001F680-\U0001F6FF"  # transport 
                           u"\U0001F1E0-\U0001F1FF"  # flags 
                           u"\U00002702-\U000027B0" 
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    
    cleaned_str = emoji_pattern.sub(r'', text)

    return cleaned_str


def clean_code(text):
    clean1 = remove_space_newline(text)
    clean2 = remove_urls(clean1)
    clean3 = remove_html(clean2)
    cleaned_str = remove_emoji(clean3)

    return cleaned_str

In [56]:
source_code_no_comments = remove_comments(source_code_comments)
tmp_json['source_code'] = source_code_no_comments

tmp_json['source_code_cleaned'] = clean_code(source_code_no_comments)
tmp_json['source_code_cleaned_comments'] = clean_code(source_code_comments)


In [57]:
print(len(tmp_json['source_code_comments']))
print(len(tmp_json['source_code']))
print(len(tmp_json['source_code_cleaned_comments']))
print(len(tmp_json['source_code_cleaned']))

197048
74793
105032
50206


In [62]:
tmp_json.keys()

dict_keys(['repo_owner', 'repo_name', 'requirements', 'readme', 'license', 'source_code_comments', 'source_code', 'source_code_cleaned_comments', 'source_code_cleaned'])

In [69]:
with open(f'{input_data_path}/{repo_owner}_{repo_name}.json', 'w') as file:
    json.dump(tmp_json, file)

In [ ]:
# delete unzip file for saving memory
# create df for further analysis with lenght of all source code fiels for hübsche grafiken :-)

In [70]:
file_path = '../data/input_data/abi_screenshot-to-code.json'
with open(file_path, 'r') as file:
    loaded_data = json.load(file)

In [73]:
print(len(loaded_data['source_code_comments']))
print(len(loaded_data['source_code']))
print(len(loaded_data['source_code_cleaned_comments']))
print(len(loaded_data['source_code_cleaned']))

197048
74793
105032
50206
